In [1]:
import simtk.openmm.app as app
import simtk.openmm as mm
import simtk.unit as unit
from simtk.openmm.app import PDBFile
import mdtraj as md
import matplotlib.pyplot as plt
import nglview as nv

_ColormakerRegistry()

**Cargar el archivo PDB del disco y agregar residuos y átomos restantes**:

In [2]:
pdb =app.PDBFile("BAK_model1_all.pdb")


**Especificar el campo de fuerza a utilizar para la simulación**:

In [3]:
forcefield =app.ForceField("amber14-all.xml", "amber14/tip3p.xml")

**Crear una descripción matematica completa del sistema por simular**:

In [4]:
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

**Crear el integrador**:

Esta línea crea el integrador empleado para avanzar las ecuaciones de movimiento. 

LangevinIntegrator: realiza la dinámica de Langevin, y lo asigna a una variable llamada integrador.

Se declaran los valores de tres parámetros que son específicos de la dinámica de Langevin: la temperatura de simulación (300 K), el coeficiente de fricción (1 ps -1 ) y el tamaño del paso (2 fs).

In [5]:
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds, 2.0*unit.femtoseconds)
integrator.setConstraintTolerance(0.00001)

**Preparar la simulación**:

- Definir la plataforma (CUDA o CPU):

In [6]:
platform = mm.Platform.getPlatformByName('CPU')
#properties = {'CudaPrecision': 'mixed'}
properties = {}

- Crear el objeto simulación:

Esta línea combina la topología molecular, el sistema y el integrador para comenzar una nueva simulación.

In [7]:
simulation = app.Simulation(pdb.topology, system, integrator, platform, properties)

In [8]:
simulation.context.setPositions(pdb.positions)

(Esta línea especifica las posiciones iniciales de los átomos para la simulación, en este caso: las posiciones que se cargaron desde el archivo PDB.)

**Minimización de energía**:

In [9]:
Edo_PreMin = simulation.context.getState(getEnergy = True)

In [10]:
Edo_PreMin.getPotentialEnergy()

Quantity(value=9444.613410471808, unit=kilojoule/mole)

In [11]:
print('Minimizing...')
simulation.minimizeEnergy(tolerance=0.1*unit.kilojoule/unit.mole, maxIterations=500)

Minimizing...


In [15]:
Edo_PosMin = simulation.context.getState(getEnergy = True, getPositions = True)

In [16]:
Edo_PosMin.getPotentialEnergy()

Quantity(value=-6142.767064739264, unit=kilojoule/mole)

In [17]:
positions = Edo_PosMin.getPositions()

**Guardar archivo PDB post-minimización**

In [19]:
PDBFile.writeFile(simulation.topology, positions, open('output_post-min.pdb', 'w'))

**Calcular el RMSD entre las estructuras pre y post minimización**

Con Hidrógenos:

#(crear objetos trayectoria)

In [20]:
PreMin = md.load("BAK_model1_all.pdb")
PostMin = md.load("output_post-min.pdb")

In [33]:
RMSD = md.lprmsd(PreMin, PostMin)

In [34]:
RMSD

array([0.17335509], dtype=float32)

Sin hidrógenos

In [26]:
lista_atomos_pesados = PreMin.topology.select("type != H")

In [35]:
RMSD_heavy_atomms = md.lprmsd(PreMin, PostMin, atom_indices = lista_atomos_pesados)

In [36]:
RMSD_heavy_atomms

array([0.17411445], dtype=float32)

**RMSD Backbone**

In [29]:
backbone = PreMin.topology.select("backbone")

In [37]:
RMSD_backbone = md.lprmsd(PreMin, PostMin, atom_indices = backbone)

In [38]:
RMSD_backbone

array([0.13880137], dtype=float32)

**Visualizar las estructuras**